In [1]:
#### relative data, where the replicates were considered, their OD relative ratio taken and the mean evaluated
#### modification : remove wells that correspond to high variability and bistability (three such wells)

#### generate permanent data for test-train split

In [2]:
import pandas as pd
import numpy as np
import random


from scipy.linalg import hadamard

import matplotlib.pyplot as plt

from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import KFold

from scipy import stats
from matplotlib import pyplot
from scipy.stats import spearmanr

import warnings
warnings.filterwarnings("ignore")

table = pd.read_pickle("~/compressed_sensingv1/realdatasets/misc/Friedman-v3/A8.pkl")
tableg = table.T
B = pd.read_pickle("~/compressed_sensingv1/realdatasets/misc/Friedman-v3/B8.pkl")

Bdf = pd.DataFrame(B)

species_ones = [] 
for i in range(8):
    species = i
    species_ones.append(list(tableg[tableg[species]==1].index))
    
    
species_zeros = []
for i in range(8):
    species = i
    species_zeros.append(list(tableg[tableg[species]==0].index))                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       



In [3]:
list_present  = pd.read_pickle("~/compressed_sensingv1/realdatasets/misc/Friedman-v3/list-present-friedman.pkl")
sst_rel = pd.read_pickle("~/compressed_sensingv1/realdatasets/misc/Friedman-v3/Friedman-data-sst-relv2.pkl")

sp_list_provided = ["Ea","Pa","Pch","Pci","Pf","Pp","Pv","Sm"] 
sst_rel2 = sst_rel.rename(columns={"Sm":0,"Pv":1,"Pp":2,"Pf":3,"Pci":4,"Pch":5,"Pa":6,"Ea":7})
sstv3 = sst_rel2[[0,1,2,3,4,5,6,7,"well index"]]

sstv4 = sstv3.fillna(0)

sstv4.head(3)

,0,1,2,3,4,5,6,7,well index
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2


In [4]:
n = 2**7
random_state = None
cv_splitsl = [3,5,7]

In [5]:
to_be_removed = {73,164,193}
list_presentv2 = [item for item in list_present if item not in to_be_removed ]

In [6]:
dtr = []
dte = [] 
for cv_splits in cv_splitsl: 
    for speciesc in range(8):
        bspecies = Bdf.T[species_ones[speciesc]].T
        bspecies["index all"] = bspecies.index
        bspecies.index = np.arange(0,2**7,1)
        abunvec = pd.DataFrame(sstv4[speciesc].values).T[species_ones[speciesc]].T
        abunvec.index = np.arange(0,2**7,1)
        listl = list(abunvec.T[list(bspecies[bspecies["index all"].isin(list_presentv2)].index)].T.index)
        data_present = list(abunvec.T[listl].T.index)
        random.shuffle(data_present)
        data_present1 = np.array(data_present)
        kf = KFold(n_splits=cv_splits)
        kf.get_n_splits(data_present1)
        KFold(n_splits=cv_splits, random_state=random_state, shuffle=True)
        count = 0 
        for train_index, test_index in kf.split(data_present1):
            X_train, X_test = data_present1[train_index], data_present1[test_index]
            rinew = list(X_train)
            rileft = list(X_test)
            y3 = abunvec.T[rinew].T[0].values  
            dtr.append([cv_splits,count,rinew,speciesc])
            dte.append([cv_splits,count,rileft,speciesc])
            count = count + 1


In [7]:
dtrf = pd.DataFradtrf = pd.DataFrame(dtr,columns=["k-fold","fold number","train data","species"])

dtef = pd.DataFrame(dte,columns=["k-fold","fold number","test data","species"])

dtrf.to_pickle("traindataRELfried.pkl")
dtef.to_pickle("testdataRELfried.pkl")


In [8]:
def flatten(l):
    return [item for sublist in l for item in sublist]

In [2]:
import pandas as pd
dtef = pd.read_pickle("testdataRELfried.pkl")

In [3]:
dtef

,k-fold,fold number,test data,species
0,3,0,"[0, 9, 5, 8, 24, 123, 34, 10, 18, 12, 72, 2]",0
1,3,1,"[125, 32, 68, 111, 33, 119, 96, 64, 20, 6, 17, 4]",0
2,3,2,"[63, 40, 66, 127, 126, 16, 3, 95, 1, 80, 48]",0
3,3,0,"[126, 0, 95, 6, 1, 17, 2, 111, 64, 16, 33, 12]",1
4,3,1,"[24, 36, 20, 72, 3, 34, 125, 5, 80, 66, 40, 63]",1
...,...,...,...,...
115,7,2,"[0, 68, 65, 119, 2]",7
116,7,3,"[24, 48, 33, 17, 111]",7
117,7,4,"[20, 18, 34, 80, 4]",7
118,7,5,"[125, 127, 64, 1, 123]",7
